# Agentic retrieval using Azure AI Search and Azure AI Agent Service

Use this notebook to create an agentic retrieval pipeline built on Azure AI Search and an Azure AI Agent.

In this walkthrough, you will:

+ Create an "earth_at_night" search index
+ Load it with documents from a GitHub URL
+ Create a knowledge source that points to searchable content.
+ Create a knowledge agent on Azure AI Search that points to a knowledge source and an LLM for intelligent query planning
+ Create a Foundry agent in Azure AI Foundry to determine when queries are needed
+ Create a Azure AI Agent tool (client) to orchestrate all requests
+ Start a chat with the agent

This notebook is referenced in [Build an agentic retrieval pipeline in Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-pipeline).

This exercise differs from the [Agentic Retrieval Quickstart](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval) in how it uses Azure AI Agent to determine whether to retrieve data from the index, and how it uses an agent tool for orchestration.

## Prerequisites

+ Azure AI Search, basic tier or higher, in [any region that supports semantic ranker](https://learn.microsoft.com/azure/search/search-region-support#azure-public-regions).

+ Azure OpenAI, and you should have an **Azure AI Developer** role assignment to create a Foundry project.

+ An [Azure AI agent and Foundry project](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal), created in the Azure AI Foundry portal, with the basic setup, used for creating the Foundry agent.

+ A deployment of a [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) in your Foundry project. This notebook uses gpt-4o-mini. We recommend 100,000 token capacity. You can find capacity and the rate limit in the model deployments list in the Azure AI Foundry portal.

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.13.7.

## Set up connections

Save the `sample.env` file as `.env` and then modify the environment variables to use your Azure endpoints. You need endpoints for:

+ Azure AI Search
+ Azure OpenAI
+ Azure AI Foundry project

You can find endpoints for Azure AI Search and Azure OpenAI in the [Azure portal](https://portal.azure.com).

You can find the project endpoint in the Azure AI Foundry portal:

1. Sign in to the [Azure AI Foundry portal](https://ai.azure.com) and open your project. 

1. In the **Overview** tile, find and copy the **Azure AI Foundry project endpoint**. 

   A hypothetical endpoint might look like this: `https://your-foundry-resource.services.ai.azure.com/api/projects/your-foundry-project`

## Load Connections

Load the environment variables to set up connections and object names.

In [4]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_endpoint = os.environ["PROJECT_ENDPOINT"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE_NAME", "earth-at-night-ks")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")

## Create search index on Azure AI Search

This steps create a search index that contains plain text and vector content. You can use any existing search index, but it must meet the [criteria for agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schmea requirement is that is has a semantic configuration, with a `default_configuration_name`.

In [2]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")


Index 'earth_at_night' created or updated successfully


## Upload sample documents

This sample uses data from NASA's Earth at Night e-book. It's retrieved from the sample data GitHub repository and passed to the search client for indexing.

In [3]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")


Documents uploaded to index 'earth_at_night'


## Create a knowledge source

This step creates a knowledge source that targets the index you previously created. In the next step, you create a knowledge agent that uses the knowledge source to orchestrate agentic retrieval.


In [5]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters
from azure.search.documents.indexes import SearchIndexClient

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_select="id,page_chunk,page_number",
    ),
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'earth-at-night-ks' created or updated successfully.


## Create a knowledge agent on Azure AI Search

This step creates a knowledge agent, which acts as a wrapper for your knowledge source and LLM deployment.

EXTRACTIVE_DATA is the default modality and returns content from your knowledge sources without generative alteration. However, this quickstart uses the ANSWER_SYNTHESIS modality for LLM-generated [answers that cite the retrieved content](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-synthesize).

In [9]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality
from azure.search.documents.indexes import SearchIndexClient

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_gpt_deployment,
    model_name=azure_openai_gpt_model,
)

output_cfg = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.EXTRACTIVE_DATA,
    include_activity=True,
)

agent = KnowledgeAgent(
    name=agent_name,
    models=[KnowledgeAgentAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name,
            include_reference_source_data=True,
            include_references=True
        )
    ],
    output_configuration=output_cfg,
)


index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_agent(agent)
print(f"Knowledge agent '{agent_name}' created or updated successfully")

Knowledge agent 'earth-search-agent' created or updated successfully


## Create an Azure AI Agent

In the Azure AI Foundry, an agent is a smart micro-service that can do RAG. The purpose of this specific agent is to decide when to send a query to the agentic retrieval pipeline.

In [7]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

list(project_client.agents.list_agents())

[]

In [15]:
instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)

print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'earth-search-agent' created or updated successfully


## Add an agentic retrieval tool to AI Agent

An end-to-end pipeline needs an orchestration mechanism for coordinating calls to the retriever and agent. The pattern described in this notebook uses a [tool](https://learn.microsoft.com/azure/ai-services/agents/how-to/tools/function-calling) for this task. The tool calls the Azure AI Search knowledge retrieval client and the Azure AI agent, and it drives the conversations with the user.

In [14]:
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder

from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=credential)

thread = project_client.agents.threads.create()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
        You must refer to references by id number
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.messages.list(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages = list(messages)
    messages.reverse()
    retrieval_result = agent_client.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[
                    KnowledgeAgentMessage(
                        role=m["role"],
                        content=[KnowledgeAgentMessageTextContent(text=m["content"])]
                    ) for m in messages if m["role"] != "system"
            ]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text

# https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/function-calling
functions = FunctionTool({ agentic_retrieval })
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset)

## Start a chat with the agent

During the chat, you use the standard Azure AI agent tool calling APIs.  We send the message with questions, and the agent decides when to retrieve knowledge from your search index using agentic retrieval.

The remaining cells take a closer look at output and show how to add another turn to the conversation.

In [18]:
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value

print("Agent response:", output.replace(".", "\n"))


Agent response: Suburban belts exhibit larger December brightening than urban cores despite higher absolute light levels downtown primarily because of holiday light displays and activities concentrated in the suburbs
 Suburban areas, with more yard space and a prevalence of single-family homes, tend to put up more holiday lights around Christmas and New Year's, resulting in a significant increase in light intensity by 20 to 50 percent in December
 In contrast, central urban areas, though brighter overall, show smaller relative increases in brightness during the holidays
 This pattern reflects cultural traditions and patterns of energy consumption, with suburban lighting displays contributing more to December brightening [refs 1, 3, 5]


Regarding the sharp visibility of the Phoenix nighttime street grid from space compared to the dimmer interstate stretches between Midwestern cities, the reason lies in urban layout and lighting distribution
 Phoenix and its metropolitan area are charac

## Review retrieval activity and results

Each retrieval response from Azure AI Search includes the unified string (grounding data from search search results), the query plan, and  reference data showing which chunks of source document contributed content to the unified string.

In [19]:
import json

retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))


Retrieval activity
[
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 2342,
    "input_tokens": 2256,
    "output_tokens": 117
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 1145,
    "knowledge_source_name": "earth-at-night-ks",
    "query_time": "2025-09-09T03:52:26.409Z",
    "count": 3,
    "search_index_arguments": {
      "search": "Why do suburban belts display larger December brightening than urban cores despite higher absolute light levels downtown?"
    }
  },
  {
    "id": 2,
    "type": "searchIndex",
    "elapsed_ms": 674,
    "knowledge_source_name": "earth-at-night-ks",
    "query_time": "2025-09-09T03:52:27.094Z",
    "count": 4,
    "search_index_arguments": {
      "search": "Why is the Phoenix nighttime street grid sharply visible from space compared to dimmer stretches of interstate between Midwestern cities?"
    }
  },
  {
    "id": 3,
    "type": "semanticReranker",
    "input_tokens": 46720
  }
]
Retrieval results
[
  {
  

## Continue the conversation...

In [21]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night? Use the retrieval tool to answer this question."
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value


print("Agent response:", output.replace(".", "\n"))


Agent response: To find lava at night, satellite instruments can be used to detect the thermal infrared signature emitted by the hot lava flows
 For example, during eruptions such as those at Mount Etna in Italy and Kilauea in Hawaii, satellites like Landsat 8 use instruments such as the Operational Land Imager (OLI) and the Thermal Infrared Sensor (TIRS) to capture images that highlight active vents and flowing lava by their heat emission
 These thermal infrared images show very hot lava as bright white or red, distinguishing it clearly from cooler surroundings and city lights


Additionally, the VIIRS Day/Night Band (DNB) on polar-orbiting satellites can detect faint light sources including the glow from lava flows at night, often combined with moonlight to enhance visibility
 Nighttime glow from lava can thus be observed as bright spots different from city lights in satellite images, making it possible to monitor volcanic activity even in darkness
 This technique is used to track la

## Review retrieval activity and results


In [22]:
retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 1694,
    "input_tokens": 2662,
    "output_tokens": 74
  },
  {
    "id": 1,
    "type": "searchIndex",
    "elapsed_ms": 617,
    "knowledge_source_name": "earth-at-night-ks",
    "query_time": "2025-09-09T03:53:13.369Z",
    "count": 17,
    "search_index_arguments": {
      "search": "How to find lava at night"
    }
  },
  {
    "id": 2,
    "type": "searchIndex",
    "elapsed_ms": 631,
    "knowledge_source_name": "earth-at-night-ks",
    "query_time": "2025-09-09T03:53:14.001Z",
    "count": 16,
    "search_index_arguments": {
      "search": "Methods to locate lava flows during nighttime"
    }
  },
  {
    "id": 3,
    "type": "semanticReranker",
    "input_tokens": 44714
  }
]
Retrieval results
[
  {
    "type": "searchIndex",
    "id": "0",
    "activity_source": 2,
    "source_data": {
      "id": "earth_at_night_508_page_46_verbalized",
      "page_chunk": "For the first time in perha

## Clean up objects and resources

If you no longer need the resources, be sure to delete them from your Azure subscription.  You can also delete individual objects to start over.

### Delete the agent

In [23]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_agent(agent_name)
print(f"Knowledge agent '{agent_name}' deleted successfully")

Knowledge agent 'earth-search-agent' deleted successfully


### Delete the knowledge source

In [24]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_knowledge_source(knowledge_source=knowledge_source_name) # This is new feature in 2025-08-01-Preview api version
print(f"Knowledge source '{knowledge_source_name}' deleted successfully.")


Knowledge source 'earth-at-night-ks' deleted successfully.


### Delete the index

In [ ]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")

Index 'earth_at_night' deleted successfully
